In [1]:
import open3d as o3d
import numpy as np
import copy as cp
import open3d.core as o3c
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation
from iteration_utilities import deepflatten

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#load pcd file, filter, downsample
pcdn = o3d.io.read_point_cloud("final_cropped.pcd")
pcdn.estimate_normals()
cl, ind = pcdn.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=0.8)

pcd = pcdn.select_by_index(ind)
pcd = pcd.voxel_down_sample(voxel_size=0.1)
pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(40)
o3d.visualization.draw_geometries([pcd])

In [3]:
# detect boundarys in pointcloud

test2 = cp.deepcopy(pcd)

tensor_pcd = o3d.t.geometry.PointCloud.from_legacy(test2)

boundarys, mask = tensor_pcd.compute_boundary_points(0.2, 100, 95)
# TODO: not good to get size of points.
print(f"Detect {boundarys.point.positions.shape[0]} bnoundary points from {tensor_pcd.point.positions.shape[0]} points.")

boundarys = boundarys.paint_uniform_color([1.0, 0.0, 0.0])
cl, ind = boundarys.to_legacy().remove_radius_outlier(2, 0.2)
boundarys = boundarys.select_by_index(ind)


o3d.visualization.draw_geometries([tensor_pcd.to_legacy().paint_uniform_color([0, 0, 0.0]), boundarys.to_legacy()])

Detect 15121 bnoundary points from 106083 points.


In [4]:
def select_points_by_label(pcd, labels):
    max_label = labels.max()
    indices ={}
    pcds = []
    for x in range(len(pcd.points)):
        label = labels[x]
        
        if label in indices:
            indices[label].append(x)
        else:
            indices[label] = [x]
        
    for x in indices.keys():
        if x >= 0:
            pcds.append(pcd.select_by_index(indices[x]))
    
    return pcds

In [11]:
pcd_cluster = cp.deepcopy(boundarys.to_legacy())

with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(
        pcd_cluster.cluster_dbscan(eps=0.45, min_points=10, print_progress=True))

clusterd_pcds = select_points_by_label(pcd_cluster, labels)
clusterd_oboxes = []
clusterd_meshes = []
for x in range(len(clusterd_pcds)):
    color = np.random.rand(3,1)
    clusterd_pcds[x].paint_uniform_color(color)
    obox = clusterd_pcds[x].get_oriented_bounding_box()
    obox.color = color
    
    mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(obox, scale=[1, 1, 0.0001])
    mesh.paint_uniform_color(color)
    mesh.compute_triangle_normals()
    mesh.compute_vertex_normals()    
    
    if (max(obox.extent)<= 10):
        clusterd_oboxes.append(obox)
        clusterd_meshes.append(mesh)

o3d.visualization.draw_geometries(clusterd_meshes+ clusterd_pcds)

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 66
Precompute neighbors.[========================================] 100%


In [6]:
def compute_transform(vector1, vector2):
    # Normalize the vectors to unit length
    vector1 = vector1 / np.linalg.norm(vector1)
    vector2 = vector2 / np.linalg.norm(vector2)

    # Compute the rotation matrix
    rotation_matrix = Rotation.align_vectors([vector1], [vector2])[0].as_matrix()

    # Compute the translation vector
    translation_vector = vector2 - np.dot(rotation_matrix, vector1)

    return translation_vector, rotation_matrix

In [7]:
def check_vector_similar_direction(vec1, vec2):
    
    # Normalize the vectors
    normalized_vec1 = np.array(vec1) / np.linalg.norm(vec1)
    normalized_vec2 = np.array(vec2) / np.linalg.norm(vec2)
    
    # Calculate the dot product
    dot_product = np.dot(normalized_vec1, normalized_vec2)
    
    if (dot_product > 0):
        return True
    else:
        return False

In [8]:
def return_normal_arrow_from_plane(mesh, pcd, bb):
    mesh_normal = np.asarray(mesh.vertex_normals)[0]
    mesh_center = mesh.get_center()
    mesh_bb = bb
    mesh_dim = np.cbrt(mesh_bb.volume()/mesh_bb.extent[2])
    
    ind = mesh_bb.get_point_indices_within_bounding_box(pcd.points)
    part_pc = pcd.select_by_index(ind)
    normals = np.asarray(part_pc.normals)
    norm_average = np.average(normals, axis=0)
    #print("Norm average:")
    #print(norm_average)
    
    #arrow = o3d.geometry.TriangleMesh.create_arrow(0.1,0.15,0.5,0.3)
    arrow = o3d.geometry.TriangleMesh.create_arrow(0.1,0.15,mesh_dim,0.3)
    arrow.paint_uniform_color([0,1,0])
    arrow_bb = arrow.get_oriented_bounding_box()
    arrow.compute_triangle_normals()
    arrow.compute_vertex_normals()
    arrow_normal = np.asarray(arrow.vertex_normals)[0]

    translation, rotation = compute_transform(mesh_normal, arrow_normal)  
    
    arrow.rotate(rotation)
    
    arrow.translate(mesh_center, False)
    
    
    arrow_normal = np.asarray(arrow.vertex_normals)[0]
    arrow_center = arrow.get_center()
    same_dir = check_vector_similar_direction(arrow_normal,norm_average)
    R_x = np.asarray([[1, 0, 0],
                       [0, -1, 0],
                       [0, 0, -1]])
    
    arrow_cp = o3d.geometry.TriangleMesh.create_arrow(0.1,0.15,mesh_dim,0.3)
    arrow_cp.paint_uniform_color([0,1,0])
    if  (same_dir):
        arrow_cp.rotate(R_x, arrow_center)
        arrow_cp.rotate(rotation)
        arrow_cp.translate(mesh_center, False)
        return arrow_cp, arrow_bb
    else:
        return arrow, arrow_bb
    

In [16]:
#test5, bbs, planes = select_points_in_bb(boundarys.to_legacy(), geometries, 0)

arrows = []
for plane, bb in zip(clusterd_meshes, clusterd_oboxes):
    
    arrow, arrow_bb = return_normal_arrow_from_plane(plane, boundarys.to_legacy(), bb)
    arrows.append(arrow)
    



o3d.visualization.draw_geometries([pcd] + arrows + [boundarys.to_legacy()])

/tmp/ipykernel_6512/927153186.py:7: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  rotation_matrix = Rotation.align_vectors([vector1], [vector2])[0].as_matrix()
